# Sumário

O presente código apresenta uma Rede Neural utilizada para previsão em fraudes de cartões de crédito estruturada do "zero". Dito isso, os processos para realização foram os seguintes:

1) Compreensão e análise do dataset: nesta etapa, notou-se que seriam utilizados dados numéricos (não havendo necessidade de codificação de variáveis) e que o dataset já estava equilibrado (todas as colunas com o mesmo número de linhas) destacando a inexistência de valores faltantes. O único problema exposto foi o desequilíbrio na distribuição da nossa variável alvo (Class), entrave esse que foi resolvido na etapa seguinte.

2) Separando a label das features e divisão do dataset em dois subsets de treinamento e teste e treinamento e avaliação: inicialmente, dividiu-se as variáveis em inputs (independentes) e target (dependente) e normalizou a distribuição das primeiras. Após isso, foi criada a primeira instância de treino e teste (que foi utilizada posteriormente na hora da avaliação do modelo sob o conjunto de teste) para não enviesar a performance do modelo com os dados sintéticos que seriam utilizados para correção do *oversampling*. Após utilização do SMOTE, estruturou-se o segundo subset de treino e teste, esse que foi utilizado nos processos de treinamento e validação do modelo (antes das previsões no conjunto de teste).

3) Definição da espinha dorsal da rede neural: aqui, foram criadas as classes (cada uma com suas respectivas funções) que estruturaram a rede neural, sendo elas:

1. Layer: nesta classe foi definida a arquitetura das camadas da Rede Neural, estabelecendo processos como: inicialização, processamento de inputs e cálculo de gradientes com relação a perda.

2. ActivationReLU e ActivationSigmoid: nestas duas classes foram definidas as funções de ativação dos neuronios que compoõem a nossa rede.

3. LossBinaryCrossEntropy: definição e implementação da função de perda, responsável por medir a disparidade entre o que esta sendo previsto e o real, isto é, o quão distante a rede está da resposta real.


4) Treinamento, validação e teste do modelo: Definição da função de treinamento e validação do modelo e, posteriormente, avaliação das previsões do modelo perante o conjunto de teste.

Nesta etapa, foi definida também a arquitetura da rede neural, seguindo a seguinte estrutura:
* Layer 1 (Input) : 31 neurônios;
* Layer 2: 124 neurônios (Hidden Layer);
* Layer 3: 2 neurônios;
* Layer 4 (Output) : Resultado final







In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file_path = '/content/creditcard.csv'
df = pd.read_csv(file_path)

print(df.shape)
print(df.info())
# Verificando se existem valores nulos
print(df.isnull().sum())
print(df['Class'].value_counts())


(284807, 31)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 2

Nota-se que há um desbalanceamento muito grande na nossa varíavel target

#Balanceando a varíavel target, estruturando e avaliando a Rede Neural

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from imblearn import under_sampling, over_sampling

# Separando a label das features
inputs = df.drop('Class', axis=1)
targets = df['Class']
# Normalizando as varíaveis independentes
x = inputs
scaler = StandardScaler()
x_new = scaler.fit_transform(x)
y = targets

# Dividindo o dataset em treino e teste (primeiro subset)
x_train, x_test, y_train, y_test = train_test_split(x_new, y, test_size=0.5, random_state=42)

smote = SMOTE(sampling_strategy = 0.3, random_state=42)
x_resampled, y_resampled = smote.fit_resample(x_new, y)

# Verificando a nova distirbuição da variavel alvo
print(pd.Series(y_resampled).value_counts())
# Dvidindo o dataset em treino e validação após o tratamento do oversampling com SMOTE
X_train_resampled, X_val_resampled, y_train_resampled, y_val_resampled = train_test_split(x_resampled, y_resampled, test_size=0.25, random_state=42)

# Definição da "espinha dorsal" da Rede Neural
class Layer:
    def __init__(self, n_inputs, n_neurons):
      # Incializacação aleatória dos pesos
        self.weights = np.random.randn(n_inputs, n_neurons) * 0.01
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.input = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues, learning_rate):
        dvalues = dvalues.reshape(-1, 1) if len(dvalues.shape) == 1 else dvalues
        self.dweights = np.dot(self.input.T, dvalues) / self.input.shape[0]
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True) / self.input.shape[0]
        self.weights -= learning_rate * self.dweights
        self.biases -= learning_rate * self.dbiases
        self.dinput = np.dot(dvalues, self.weights.T)

class ActivationReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinput = dvalues.copy()
        self.dinput[self.output <= 0] = 0

class ActivationSigmoid:
    def forward(self, inputs):
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinput = dvalues * (self.output * (1 - self.output))

class LossBinaryCrossEntropy:
    def calculate(self, outputs, targets):
        outputs = np.clip(outputs, 1e-7, 1 - 1e-7)
        return -np.mean(targets * np.log(outputs) + (1 - targets) * np.log(1 - outputs))

    def backward(self, outputs, targets):
        outputs = np.clip(outputs, 1e-7, 1 - 1e-7)
        return (outputs - targets) / (outputs * (1 - outputs))

# Definindo a função de Treinamento e Avaliação, bem como a estrutura da rede neural
def train_and_evaluate(X_train_resampled, y_train_resampled, X_val_resampled, y_val_resampled, learning_rate, epochs, batch_size):
    dense1 = Layer(X_train_resampled.shape[1], 124)
    activation1 = ActivationReLU()
    dense2 = Layer(124, 2)
    activation2 = ActivationReLU()
    dense3 = Layer(2, 1)
    activation3 = ActivationSigmoid()
    loss_function = LossBinaryCrossEntropy()

    for epoch in range(epochs):
        for start in range(0, X_train_resampled.shape[0], batch_size):
            batch_X = X_train_resampled[start:start + batch_size]
            batch_y = y_train_resampled.iloc[start:start + batch_size].values.reshape(-1, 1)

            dense1.forward(batch_X)
            activation1.forward(dense1.output)
            dense2.forward(activation1.output)
            activation2.forward(dense2.output)
            dense3.forward(activation2.output)
            activation3.forward(dense3.output)

            loss_value = loss_function.calculate(activation3.output, batch_y)
            dvalues = loss_function.backward(activation3.output, batch_y)
            activation3.backward(dvalues)
            dense3.backward(activation3.dinput, learning_rate)
            activation2.backward(dense3.dinput)
            dense2.backward(activation2.dinput, learning_rate)
            activation1.backward(dense2.dinput)
            dense1.backward(activation1.dinput, learning_rate)

        # Avaliar no conjunto de validação
        dense1.forward(X_val_resampled)
        activation1.forward(dense1.output)
        dense2.forward(activation1.output)
        activation2.forward(dense2.output)
        dense3.forward(activation2.output)
        activation3.forward(dense3.output)

        val_loss = loss_function.calculate(activation3.output, y_val_resampled.values.reshape(-1, 1))

        if epoch % 100 == 0:
            print(f"Epoch {epoch}: Loss = {val_loss:.4f}")

    return dense1, activation1, dense2, activation2, dense3, activation3

# Configurando o número de epochs (épocas), learning rate e batch size
learning_rates = [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2]
epochs = 400
batch_size = 32
best_model = None
best_f1 = 0
for lr in learning_rates:
    print(f"\nTreinando com learning_rate={lr}")
    model = train_and_evaluate(X_train_resampled, y_train_resampled, X_val_resampled, y_val_resampled, lr, epochs, batch_size)

    dense1, activation1, dense2, activation2, dense3, activation3 = model

    # Avaliar no conjunto de validação
    dense1.forward(X_val_resampled)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    dense3.forward(activation2.output)
    activation3.forward(dense3.output)

    y_pred_val = (activation3.output >= 0.5).astype(int)
    accuracy_val = accuracy_score(y_val_resampled, y_pred_val)
    precision_val = precision_score(y_val_resampled, y_pred_val)
    recall_val = recall_score(y_val_resampled, y_pred_val)
    f1_val = f1_score(y_val_resampled, y_pred_val)

    print(f"Validation: Accuracy = {accuracy_val:.4f}, Precision = {precision_val:.4f}, Recall = {recall_val:.4f}, F1 = {f1_val:.4f}")

    # Printando previsões (no conjunto de validação) para auditoria
    print(f"Previsões: {y_pred_val.flatten()[:10]}")

    # Selecionando o modelo que obteve melhor performance no treinamento
    if f1_val > best_f1:
        best_f1 = f1_val
        best_model = model

# Prever resultados, no modelo de teste, utilizando o modelo com melhor performance
dense1, activation1, dense2, activation2, dense3, activation3 = best_model

dense1.forward(x_test)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)
dense3.forward(activation2.output)
activation3.forward(dense3.output)

y_pred_test = (activation3.output >= 0.5).astype(int)
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test)
recall_test = recall_score(y_test, y_pred_test)
f1_test = f1_score(y_test, y_pred_test)
# Avaliando a performance
print("\nResultados finais no conjunto de teste:")
print(f"Accuracy = {accuracy_test:.4f}, Precision = {precision_test:.4f}, Recall = {recall_test:.4f}, F1 = {f1_test:.4f}")
print(f"Previsões finais: {y_pred_test.flatten()[:10]}")


Class
0    284315
1     85294
Name: count, dtype: int64

Treinando com learning_rate=0.001
Epoch 0: Loss = 0.5441
Epoch 100: Loss = 0.0069


<ipython-input-2-b2baee493055>:53: RuntimeWarning: overflow encountered in exp
  self.output = 1 / (1 + np.exp(-inputs))


Epoch 200: Loss = 0.0042
Epoch 300: Loss = 0.0035
Validation: Accuracy = 0.9994, Precision = 0.9976, Recall = 1.0000, F1 = 0.9988
Previsões: [1 1 0 0 0 0 1 0 0 0]

Treinando com learning_rate=0.005
Epoch 0: Loss = 0.1207
Epoch 100: Loss = 0.0031


<ipython-input-2-b2baee493055>:53: RuntimeWarning: overflow encountered in exp
  self.output = 1 / (1 + np.exp(-inputs))


Epoch 200: Loss = 0.0028
Epoch 300: Loss = 0.0029
Validation: Accuracy = 0.9996, Precision = 0.9984, Recall = 1.0000, F1 = 0.9992
Previsões: [1 1 0 0 0 0 1 0 0 0]

Treinando com learning_rate=0.01
Epoch 0: Loss = 0.1042


...

Epoch 100: Loss = 0.1754
Epoch 200: Loss = 0.1754
Epoch 300: Loss = 0.1754
Validation: Accuracy = 0.9557, Precision = 0.9986, Recall = 0.8098, F1 = 0.8944
Previsões: [1 1 0 0 0 0 1 0 0 0]

Treinando com learning_rate=0.2
Epoch 0: Loss = 0.0185


<ipython-input-2-b2baee493055>:53: RuntimeWarning: overflow encountered in exp
  self.output = 1 / (1 + np.exp(-inputs))


Epoch 100: Loss = 3.7280
Epoch 200: Loss = 3.7280
Epoch 300: Loss = 3.7280


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation: Accuracy = 0.7687, Precision = 0.0000, Recall = 0.0000, F1 = 0.0000
Previsões: [0 0 0 0 0 0 0 0 0 0]

Resultados finais no conjunto de teste:
Accuracy = 0.9997, Precision = 0.8632, Recall = 1.0000, F1 = 0.9266
Previsões finais: [1 0 0 0 0 0 0 0 0 0]


# Avaliação dos resultados

Antes de analisarmos a performance, é válido ressaltar os respectivos parâmetros utilizados no treinamento:
1. Número de épocas: 400;
2. Learning rate: 0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2;
3. Batch size: 32

Agora, vamos analisar cada cenário:

* Treinamento com Learning rate = 0.001 e learning rate = 0.005;

Notou-se não somente um acentuado decréscimo na taxa de perda (com poucas iterações) como também um elevado valor para todas as métricas de performance, caracterizando um possível overfitting.

* Treinamento com Learning rate = 0.01 e Learning rate = 0.05;

Novamente, nota-se um comportamento análogo ao caso anterior (baixas taxas de perda e altos valores para as métricas de avaliação), caracterizando também um possível overfitting.

* Treinamento com Learning rate = 0.1;

Aqui, apresenta-se um cenário tipico de underfitting haja vista que, o modelo apresenta uma elevação na taxa de perda ao longo das iterações e baixos valores em métricas como precisão e f1, indicando que o modelo está cometendo muitos erros de previsão.

* Treinamento com Learning rate = 0.15;

Neste cenário, não há evidências claras de overfitting (como nos exemplos anteriores) mesmo com a boa performance do modelo, o que já o descaracterizaria como underfitting.

* Treinamento com Learning Rate = 0.2;

Em específico, neste caso, o modelo apresentou sérias dificuldades para se ajustar aos dados, fato esse que fez com que esse treinamento fosse descartado da análise.

Por fim, tomou-se o modelo com o maior F1-score ao longo do treinamento e o avaliou com relação às análises no conjunto de teste (que não continha amostras sintéticas). Mediante a análise das métricas de avaliação, pode-se concluir que o modelo apresentou um resultado satisfatório nas previsões realizadas.



